# Introduction to the `Hamiltonian` Module in Paddle Quantum
*Copyright (c) 2023 Institute for Quantum Computing, Baidu Inc. All Rights Reserved.*

The `Hamiltonian` module in Paddle Quantum is a module related to Hamiltonians, which allows users to quickly create custom Hamiltonians in the form of Pauli strings and obtain various related information about the Hamiltonians.

## Creation of Hamiltonians

The Hamiltonian of a quantum system is an observable related to the total energy of the system and has important applications in quantum simulation algorithms such as the [Variational Quantum Eigensolver](https://qml.baidu.com/tutorials/quantum-simulation/variational-quantum-eigensolver.html) and [Hamiltonian Simulation with Product Formula](https://qml.baidu.com/tutorials/quantum-simulation/hamiltonian-simulation-with-product-formula.html). In quantum computing, the Hamiltonian needs to be represented in the form of a sum of tensor products of Pauli operators that can be processed by a quantum computer. For example, the following Hamiltonian is represented in this form:

$$
H =  Z \otimes Z \otimes I +  I \otimes X \otimes Z
\tag{1}
$$

In Paddle Quantum, users can create Hamiltonians in the form of Pauli strings by defining a list of the coefficients and corresponding Pauli operators for each term in the Hamiltonian. The identity operator $I$ can be omitted in each Pauli operator term. Let's try to create the Hamiltonian above.

In Paddle Quantum, users can create Hamiltonians by defining a list that contains information about multiple tensor products of Pauli operators. Each element of the list should include the coefficient and the string representation of the corresponding Pauli operator. For example, the first term in the Hamiltonian $H$ above is $Z \otimes Z \otimes I$, with a coefficient of $1$ and a corresponding string representation of 'Z0, Z1'，where 'Z0' and 'Z1' indicate that the Pauli operator acts on the 0-th and 1st qubits, respectively. Here, the identity operator $I$ can be omitted. Let's try to create the Hamiltonian $H$ above.




In [12]:
import numpy as np
import paddle_quantum
from paddle_quantum import Hamiltonian
from paddle_quantum.qinfo import random_hamiltonian_generator
from paddle_quantum.trotter import  get_1d_heisenberg_hamiltonian


h = Hamiltonian([[1,'Z0, Z1'], [1,'X1, Z2']])
print(h)

1.0 Z0, Z1
1.0 X1, Z2


## Hamiltonian Operations

The `Hamiltonian` class in Paddle Quantum supports operations such as automatic merging of like terms, addition and subtraction, coefficient multiplication, indexing, and splitting, as shown below:

In [13]:
h = Hamiltonian([[0.5,'Z0, Z1'], [0.5,'Z1, Z0'], [1,'X1, Z2']], compress = True)
print(h)

1.0 Z0, Z1
1.0 X1, Z2


In [14]:
h1 = Hamiltonian([[1,'Z0, Z1']])
h2 = Hamiltonian([[1 ,'X1, Z2']])
h = h1 + h2
print('h1 + h2: \n', h)
print('h1 * 2: ', h1 * 2)
print('h1: ', h[0])

h1 + h2: 
 1.0 Z0, Z1
1.0 X1, Z2
h1 * 2:  2.0 Z0, Z1
h1:  1.0 Z0, Z1


## Common Methods

The built-in `decompose_pauli_words()` and `decompose_with_sites()` methods in `Hamiltonian` can extract the various Pauli operators and their corresponding coefficients in the Hamiltonian and decompose them into a more convenient form. Among them, decompose_pauli_words() decomposes the Hamiltonian into two parts, a coefficient list and a corresponding Pauli string list. A Pauli string representation of a $n$-qubit Pauli operator is a string of $n$ characters derived from the set $\{'I', 'X', 'Y', 'Z'\}$. For example, the Pauli string representation of the Pauli operator $Z \otimes Z \otimes I$ is 'ZZI'; decompose_with_sites() decomposes the Hamiltonian into three parts, which are the coefficient list, the simplified form of the Pauli string list, and the list of qubit indices on which they respectively act. For example, the Pauli string representation of the Pauli operator $I \otimes X \otimes Z$ is 'IXZ', which further simplifies to 'XZ', and is identified by the list [1, 2] to indicate the qubit indices on which 'X' and 'Z' respectively act.






In [15]:
print('Pauli words decomposion：', h.decompose_pauli_words())
print('Pauli with sites decomposion：', h.decompose_with_sites())

Pauli words decomposion： ([1.0, 1.0], ['ZZI', 'IXZ'])
Pauli with sites decomposion： ([1.0, 1.0], ['ZZ', 'XZ'], [[0, 1], [1, 2]])


In addition, the `construct_h_matrix()` method built in `Hamiltonian` can also create its matrix form in the Pauli Z basis.

In [16]:
h1.construct_h_matrix()

matrix([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j, -1.+0.j,  0.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j, -1.+0.j,  0.+0.j],
        [ 0.+0.j,  0.+0.j,  0.+0.j,  1.+0.j]], dtype=complex64)

## Other Ways to Create Hamiltonians

In addition to using `Hamiltonian`, users can also use the `random_hamiltonian_generator` function in the `qinfo` module of Paddle Quantum to generate a random Hamiltonian. For example, users can generate a Hamiltonian that acts on 3 qubits and has 4 mutually non-commuting Pauli operators:

In [19]:
h_random = random_hamiltonian_generator(num_qubits= 3, terms= 4)
print(h_random)

-0.7669177644685217 Z1, X2
-0.34974226780994044 Y1, Y2
0.898612346022807 Y0
0.9923126285950277 X0, X1, X2


Users can also use the `get_1d_heisenberg_hamiltonian` function in the `trotter` module of Paddle Quantum to create a Hamiltonian for a one-dimensional Heisenberg chain and simulate the spin dynamics, as shown in [Simulating the Spin Dynamics on a Heisenberg Chain](https://qml.baidu.com/tutorials/quantum-simulation/simulate-the-spin-dynamics-on-a-heisenberg-chain.html):

In [20]:
h_heisenberg = get_1d_heisenberg_hamiltonian(length=5, j_x=1, j_y=2, j_z=2, h_z=2 * np.random.rand(5) - 1, periodic_boundary_condition=False)
print('System hamiltonian：')
print(h_heisenberg)

System hamiltonian：
1.0 X0, X1
2.0 Y0, Y1
2.0 Z0, Z1
1.0 X1, X2
2.0 Y1, Y2
2.0 Z1, Z2
1.0 X2, X3
2.0 Y2, Y3
2.0 Z2, Z3
1.0 X3, X4
2.0 Y3, Y4
2.0 Z3, Z4
-0.4816758416088862 Z0
0.713595033376984 Z1
0.5129840224089792 Z2
-0.5857705588538953 Z3
0.8340974367670304 Z4
